In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
import torchio as tio

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from utils import load_module
from data_processing import *
from transforms import *
from evaluators import *
from loggers.wandb_logger import to_wandb


if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

ModuleNotFoundError: No module named 'wandb'

In [ ]:
prediction_name = "cv1-ens2-flips-majority"

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern="ground_truth.*", image_name="ground_truth", 
                image_constructor=tio.LabelMap, label_values={"lesion": 1}),
])
transform = CustomOneHot()
dataset = SubjectFolder("X:/Datasets/MSSEG2_processed/", subject_path="", subject_loader=subject_loader, transforms=transform)

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern=f"{prediction_name}.*", image_name=prediction_name, 
                image_constructor=tio.LabelMap, label_values={"lesion": 1}),
])
dataset.load_additional_data('X:/Predictions/MSSEG2/', subject_loader)

In [ ]:
seg_eval = InstanceSegmentationEvaluator(prediction_name, 'ground_truth')
evaluation = seg_eval(dataset)
evaluation['summary_stats'] = evaluation['summary_stats'].to_dataframe()

In [ ]:
evaluation['subject_stats']

In [ ]:
evaluation['summary_stats']

In [ ]:
import wandb
wandb.init(project="msseg2-evaluation", dir="X:/wandb/", name=prediction_name)
wandb.log(to_wandb(evaluation))
wandb.finish()